In [37]:
from dotenv import load_dotenv
import os
from langchain.llms import OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.elastic_vector_search import ElasticVectorSearch
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.llms import OpenAI

load_dotenv()
openai_api_key = os.environ['OPENAI_API_KEY']

llm = OpenAI(temperature=0.9)

In [38]:
with open("./docs/solenoid-valve-types.html") as f:
    state_of_the_union = f.read()
text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
texts = text_splitter.split_text(state_of_the_union)

embeddings = OpenAIEmbeddings()

docsearch = Chroma.from_texts(texts, embeddings, metadatas=[{"source": str(i)} for i in range(len(texts))])

Created a chunk of size 2505, which is longer than the specified 2000
Created a chunk of size 2201, which is longer than the specified 2000
Created a chunk of size 3391, which is longer than the specified 2000
Using embedded DuckDB without persistence: data will be transient


In [39]:
query = "What is a solenoid valve?"
docs = docsearch.similarity_search(query)

chain = load_qa_with_sources_chain(OpenAI(temperature=1), chain_type="stuff")
chain({"input_documents": docs, "question": query}, return_only_outputs=True)